# MEU205 - examen du 16 décembre 2021

---

* Durée de l'épreuve 1h30.
* Le sujet est compososé de deux exercices **indépendants**.
* Tous les documents ainsi que les calculatrices et les **téléphones portables** sont interdits.
* La communication entre les étudiants est interdite.
* Toutes vos fonctions doivent être commentées selon la norme de `python`.
---

# Exercice 1 - Une classe pour $\mathbb{Z}/p\mathbb{Z}$

Dans cet exercice, nous construisons une classe permettant de manipuler les entiers modulo $p$ ($p$ étant un entier donné) comme les entiers. C'est-à-dire que nous allons surcharger tous les opérateurs nécessaires.

Afin de vous aider dans la création de la classe, vous pouvez utiliser le décorateur `@debug` créé dans la cellule suivante. Une fois votre classe proprement codée, vous pourrez supprimer ce décorateur...

In [1]:
import math
import operator

def debug(func):
    def inner(*args, **kwargs):
        print(f"\t\t-> running {func.__name__} {args[1:]}")
        return func(*args, **kwargs)
    return inner

L'algorithme d'Euclide étendu permet de calculer une relation de Bezout entre deux entiers relatifs. Nous rappelons que, si $a$ et $b$ sont deux entiers relatifs dont le pgcd vaut $\operatorname{pgcd}(a, b)$, alors il existe deux entiers relatifs $u$ et $v$ tels que
$$ au + bv = \operatorname{pgcd}(a, b).$$
L'unicité des entiers $u$ et $v$ n'est pas assurée, mais l'algorithme d'Euclide étendu permet de trouver un couple $(u, v)$ vérifiant cette relation. Voici cet algorithme écrit en pseudo-langage :

```
euclide(a, b)
| u := 1
| v := 0
| s := 0
| t := 1
| tant que b est strictement positif, faire {
|     q := quotient de la division entière de a par b
|     a_temp := a
|     a := b
|     b := a_temp - q*b
|     u_temp := u
|     u := s
|     s := u_temp - q*s
|     v_temp := v
|     v := t
|     t := v_temp - q*t
| }
| retourne a, u, v
```

**Question 1 (Algorithme d'Euclide étendu)**

> 1. Programmez une fonction `euclide` qui prend deux entiers $a$ et $b$ en paramètres et qui retourne trois entiers, correspondant au $\operatorname{pgcd}(a, b)$ et à $u$ et $v$ tel que $au+bv=\operatorname{pgcd}(a, b)$.
> 2. Vérifiez que votre algorihtme fonctionne bien sur les entiers $15$ et $12$ puis sur les entiers $465$ et $1455$ :
$$ 15-12=3=\operatorname{pgcd}(15, 12), \qquad -25*465+8*1455=15=\operatorname{pgcd}(465, 1455).$$

In [16]:
def euclide(a, b):
    u = 1
    v = 0
    s = 0
    t = 1
    while b>0:
        q =  a // b
        a_temp = a
        a = b
        b = a_temp - q*b
        u_temp = u
        u = s
        s = u_temp - q*s
        v_temp = v
        v = t
        t = v_temp - q*t
    return  a, u, v

In [17]:
print(euclide(15,12))
print(euclide(465,1455))

(3, 1, -1)
(15, -25, 8)


## Quelques rappels sur l'anneau $\mathbb{Z}/p\mathbb{Z}$

L'anneau $\mathbb{Z}/p\mathbb{Z}$ est un ensemble fini à $p$ éléments qui permet de travailler avec les entiers modulo $p$. L'entier $p$ est appelé **la caractéristique** de l'anneau. Nous noterons ces $p$ éléments comme les entiers suivi de $[p]$
$$ \mathbb{Z}/p\mathbb{Z} = \Bigl\lbrace 0[p], 1[p], 2[p], \ldots, (p-1)[p]\Bigr\rbrace.$$
Cet anneau est muni des opérations $+$ et $*$ définies par
$$
a[p] + b[p] = c[p] \text{ avec $c$ le reste de la division euclidienne de $a+b$ par $p$} \quad (c=(a+b)\%p), 
$$
et
$$
a[p] * b[p] = c[p] \text{ avec $c$ le reste de la division euclidienne de $a*b$ par $p$} \quad (c=(a*b)\%p), 
$$

Tous les éléments de $\mathbb{Z}/p\mathbb{Z}$ ne sont pas inversibles : nous avons la propriété suivante
$$
a[p] \text{ est inversible dans } \mathbb{Z}/p\mathbb{Z} \iff
\operatorname{pgcd}(a, p) = 1.
$$
En effet, dans le cas où $a$ et $p$ sont premiers entre eux ($\operatorname{pgcd}(a, p)=1$), la relation de Bezout donne l'existence de entiers $u$ et $v$ tels que $ au + pv = 1$. Ainsi, $u[p]$ est l'inverse de $a[p]$.

## La classe `ZpZ`

La classe `ZpZ` proposée dans la cellule suivante doit nous permettre de représenter les anneaux de la forme $\mathbb{Z}/p\mathbb{Z}$ pour $p$ un entier quelconque. Cette classe contient actuellement deux attributs (censés être privés) `_p` et `_lst_inv` :
- l'attribut `_p` permet de stocker la valeur de la caractéristique $p$ de l'anneau $\mathbb{Z}/p\mathbb{Z}$ ;
- l'attribut `_lst_inv` est la liste vide. Elle permettra de stocker la liste des inverses de tous les éléments inversibles dans l'anneau.

**Question 2 (les attributs)**

> - A l'aide du décorateur `@property`, créez un attribut de type `getter` (en lecture seule) nommé `p`. Cet attribut permettra d'utiliser la valeur de $p$ sans pouvoir la modifier.
> - modifiez la ligne 27 pour que la liste des inverses `_lst_inv` deviennent une liste de taille $p$ dont le premier élément vaut $0$ et tous les autres `None`.

In [27]:
class ZpZ:
    """
    Une classe pour les entiers modulo p
    
    Parameters
    ----------
    
    p: int
        entier strictement positif égal à la caractéristique de l'anneau
        
    Examples
    --------
    
    >>> K = ZpZ(7)  # anneau Z/7Z
    >>> print(K)    # affichage
    >>> e = K(2)    # l'élément 2[7] de l'anneau Z/7Z
    >>> print(e)    # affichage
    
    """
    @debug
    def __init__(self, p):
        if not isinstance(p, int):
            raise ValueError(f"Erreur dans ZpZ : un entier est requis ({type(p)})")
        if p <= 0:
            raise ValueError(f"Erreur dans ZpZ : un entier strictement positif est requis ({p})")
        self._p = p
        self._lst_inv = [0] + (self._p - 1)*[None]
        
    @property
    def get_p(self):
        return self._p

    def __str__(self):
        """surcharge de l'opérateur print()"""
        return f"Z/{self._p}Z"

    def __repr__(self):
        """surcharge de l'opérateur print()"""
        return self.__str__()

    def __call__(self, k):
        """
        surcharge de l'opérateur ()
        
        Parameters
        ----------
        
        k: int
            un entier k
            
        Returns
        -------
        
        e: ZpZ_elem
            l'élément k[p] de Z/pZ
        """
        if not isinstance(k, int):
            raise ValueError(f"Erreur pour créer un élément : un entier est requis ({type(k)})")
        return ZpZ_elem(k, self)
    
    def __iter__(self):
        """
        Itérateur sur les éléments de l'anneau Z/pZ

        Examples
        --------
        
        >>> K = ZpZ(7)
        >>> for e in K:
        >>>    print(e)
        """
        return ZpZ_iterator(self)

    def table_operation(self, operatorname):
        """
        Affiche la table d'opération
        
        Parameters
        ----------
        
        operatorname: str
            une des chaines de caractères '+', '-', '*', '/'
        """
        if operatorname == '+':
            op = operator.add
        elif operatorname == '*':
            op = operator.mul
        elif operatorname == '-':
            op = operator.sub
        elif operatorname == '/':
            op = operator.truediv
        else:
            raise ValueError(f"L'opérateur {operatorname} n'existe pas dans table_operation()")
        t_p = int(math.log10(self._p)) + 1  # taille de p (nombre de caractères)
        l_c = 2*t_p+4                      # taille d'une case
        
        def case_1(s):
            return ' '*(l_c//2) + s + ' '*(l_c//2-1)
        def affiche_case(x):
            fmt = f'{t_p}d'                    # format pour afficher les entiers
            return f" {x:{fmt}} "

        blanc = case_1(' ')                # case blanche
        tiret = case_1('-')                # case avec un tiret
        print('\n'+"="*(l_c*(self._p+1)+1))
        print(case_1(operatorname) + '|', end='')
        for ei in self:
            print(affiche_case(ei), end='')
        print('\n'+"-"*(l_c*(self._p+1)+1))
        for ei in self:
            print(affiche_case(ei) + '|', end='')
            for ej in self:
                try:
                    print(affiche_case(op(ei, ej)), end='')
                except:
                    print(tiret, end='')
            print('')
        print("="*(l_c*(self._p+1)+1))
        

        

class ZpZ_iterator:
    ''' Iterator class '''
    def __init__(self, ring):
        self._ring = ring
        self._index = 0

    def __next__(self):
        '''Returns the next element'''
        if self._index < self._ring._p:
            self._index += 1
            return self._ring(self._index-1)
        raise StopIteration

## La classe `ZpZ_elem`

La classe `ZpZ_elem` présentée dans la cellule suivante doit nous permettre de coder et d'utiliser les éléments des anneaux `ZpZ`. Une instance de la classe `ZpZ_elem` contient deux attributs :
- un élément de la classe `ZpZ` noté `_ring` qui sera l'anneau d'appartenance de l'élément ;
- un entier `_k` dont la valeur doit être comprise entre $0$ et $p-1$ où $p$ est la caractéristique de l'anneau `_ring`.

In [107]:
class ZpZ_elem:
    """
    Une classe pour les éléments de Z/pZ
    
    Parameters
    ----------
    
    k: int
        un entier (pas nécessairement entre 0 et p-1)
        
    ring: ZpZ
        l'anneau Z/pZ
        
    Examples
    --------
    
    >>> K = ZpZ(7)
    >>> a, b = K(2), K(4)
    >>> print(f"a = {a}, b = {b}")   -> a = 2[7], b = 4[7]
    >>> print(f"a+b = {a+b}")        -> a+b = 6[7]
    >>> print(f"a-b = {a-b}")        -> a-b = 5[7]
    >>> print(f"a*b = {a*b}")        -> a*b = 1[7]
    >>> print(f"a/b = {a/b}")        -> a/b = 4[7]
    """
    def __init__(self, k, ring):
        if not isinstance(ring, ZpZ):
            raise ValueError(f"Erreur dans la création de l'élément ({type(ring)})")
        self._ring = ring
        self._k = k % self._ring._p

    def __str__(self):
        """surcharge de l'opérateur print()"""
        return f"{self._k}[{self._ring._p}]"
    
    def __repr__(self):
        """surcharge de l'opérateur print()"""
        return self.__str__()
    
    def __format__(self, spec):
        if spec == "":
            return self.__str__()
        return f"{self._k:{spec}}[{self._ring._p:{spec}}]"
        
    def __neg__(self):
        """surcharge de l'opérateur - (opposé)"""
        return ZpZ_elem(-self._k,self._ring)

    
    def __add__(self, other):
        """surcharge de l'opérateur + (addition)"""
        if type(other) == int:
            return ZpZ_elem((self._k + other),self._ring)
        if self._ring._p == other._ring._p:
            return ZpZ_elem((self._k + other._k ),self._ring)
        else:
            raise ValueError(f"Les elements ne sont pas dans le même anneau")
    
    def __radd__(self, other):
        """surcharge de l'opérateur + à droite (addition)"""
        return self.__add__(other)
    
    def __sub__(self, other):
        """surcharge de l'opérateur - (soustraction)"""
        return self.__add__(-other)
    
    def __rsub__(self, other):
        """surcharge de l'opérateur - à droite (soustraction)"""
        return -self.__sub__(other)
    
    def _compute_inverse(self):
        """calcule l'inverse de l'élément s'il n'a pas encore été calculé"""
        a, u, v = euclide(self._k,self._ring._p)
        if a == 1:
            self._ring._lst_inv[self._k] = u % self._ring._p
            self._ring._lst_inv[u % self._ring._p] = self._k
        else:
            self._ring._lst_inv[self._k] = 0
        
        return self._ring._lst_inv

    @property
    def is_invertible(self):
        """
        booléen 
        - True si l'élément est inversible
        - False sinon
        """
        if self._ring._lst_inv[self._k] is None:  # l'inverse n'a pas encore été calculé
            self._compute_inverse()
        return self._ring._lst_inv[self._k] > 0

    def __inv__(self):
        """retourne l'inverse de l'élément s'il est inversible (erreur sinon)"""
        if not self.is_invertible:
            raise ValueError(f"{self._k} n'est pas inversible dans {self._ring}")
        return self.__class__(self._ring._lst_inv[self._k], self._ring)
    
    def __mul__(self, other):
        """surcharge de l'opérateur * (multiplication)"""
        if type(other) == int:
            return ZpZ_elem((self._k * other),self._ring)
        if self._ring._p == other._ring._p:
            return ZpZ_elem((self._k * other._k ),self._ring)
        else:
            raise ValueError(f"Les elements ne sont pas dans le même anneau")
    
    def __rmul__(self, other):
        """surcharge de l'opérateur * à droite (multiplication)"""
        return self.__mul__(other)
    
    def __truediv__(self, other):
        """surcharge de l'opérateur / (division)"""
        if isinstance(other, int):
            other = self.__class__(other, self._ring)
        return self.__mul__(other.__inv__())

    def __rtruediv__(self, other):
        """surcharge de l'opérateur / à droite (division)"""        
        return self.__inv__().__mul__(other)
    
    def __pow__(self, other):
        """surcharge de l'opérateur ** (puissance)"""
        # A MODIFIER
        return self 

**Question 3 (surcharge de l'opérateur `__str__()`**

> Surchargez l'opérateur `__str__()` afin que l'affichage de l'élément soit conforme à l'exemple ci-dessous :
> ```python
> K = ZpZ(7)
> print(K(2))
>
>          2[7]
> ```
> Testez votre fonction en exécutant la cellule suivante.

In [115]:
K = ZpZ(7)
print(K(2), K(9), K(-5))

		-> running __init__ (7,)
2[7] 2[7] 2[7]


**Question 4 (surcharge des opérateurs d'opérations)**

> 1. Surchargez l'opérateur `__add__` afin qu'il retourne l'élément correspondant à la somme de l'élément `self` et de l'élément `other` (vous veillerez à vérifier que les deux éléments sont dans le même anneau). Modifiez votre code afin qu'il soit possible d'ajouter un nombre entier à un élément de $\mathbb{Z}/p\mathbb{Z}$. Testez votre code en exécutant la cellule suivante.
> 2. Surchargez l'opérateur `__neg__` afin qu'il retourne l'élément correspondant à l'opposé de l'élément `self`. Vérifiez que votre code permet alors de calculer la différence entre deux nombres en calculant et en affichant $2[7] - 5[7] = 4[7]$.
> 3. Surchargez l'opérateur `__mul__` afin qu'il retourne l'élément correspondant au produit de l'élément `self` et de l'élément `other` (vous veillerez à vérifier que les deux éléments sont dans le même anneau). Modifiez votre code afin qu'il soit possible de multiplier un nombre entier à un élément de $\mathbb{Z}/p\mathbb{Z}$. Testez votre code calculant et en affichant le produit $4[7]$ et de $2[7]$.
> 4. Pour surcharger l'opérateur de division, vous devez seulement implémentez la fonction `_compute_inverse()`. Cette fonction doit utiliser la fonction `euclide` pour calculer le $\operatorname{pgcd}$ de l'entier `self._k` et de la caractéristique de l'anneau `self._ring.p`. Si ce pgcd vaut 1, alors l'élément est inversible d'inverse $u[p]$ ($u$ étant la valeur retourné par l'algorithme d'Euclide modulo $p$). Vous devez alors modifier la liste `self._ring._lst_inv` de la façon suivante :
>   - si l'élément `self._k` est inversible d'inverse `u`, `self._ring._lst_inv[self._k]` doit prendre la valeur `u` et `self._ring._lst_inv[u]` doit prendre la valeur `self._k` ;
>   - si l'élément `self._k` n'est pas inversible, `self._ring._lst_inv[self._k]` doit prendre la valeur 0.
>
> Vérifiez que votre code fonctionne correctement en calculant et en affichant $4[7] / 2[7] = 2[7]$.

In [116]:
K2 = ZpZ(9)
a, b, c, d = K(2), K(3), 3, K2(3)
print(1/b)
print(a+c)
print(a+d) # doit retourner un message d'erreur car K et K2 ne sont pas le même anneau.

		-> running __init__ (9,)
5[7]
5[7]


ValueError: Les elements ne sont pas dans le même anneau

**Question 5 (Vérification)**

> Afin de vérifier que toutes les opérations sont bien codées, exécutez la cellule suivante qui affiche les tables d'opérations pour $\mathbb{Z}/6\mathbb{Z}$.

In [117]:
K = ZpZ(6)
for op in ['+', '*', '-', '/']:
    K.table_operation(op)

		-> running __init__ (6,)

   +  | 0[6]  1[6]  2[6]  3[6]  4[6]  5[6] 
-------------------------------------------
 0[6] | 0[6]  1[6]  2[6]  3[6]  4[6]  5[6] 
 1[6] | 1[6]  2[6]  3[6]  4[6]  5[6]  0[6] 
 2[6] | 2[6]  3[6]  4[6]  5[6]  0[6]  1[6] 
 3[6] | 3[6]  4[6]  5[6]  0[6]  1[6]  2[6] 
 4[6] | 4[6]  5[6]  0[6]  1[6]  2[6]  3[6] 
 5[6] | 5[6]  0[6]  1[6]  2[6]  3[6]  4[6] 

   *  | 0[6]  1[6]  2[6]  3[6]  4[6]  5[6] 
-------------------------------------------
 0[6] | 0[6]  0[6]  0[6]  0[6]  0[6]  0[6] 
 1[6] | 0[6]  1[6]  2[6]  3[6]  4[6]  5[6] 
 2[6] | 0[6]  2[6]  4[6]  0[6]  2[6]  4[6] 
 3[6] | 0[6]  3[6]  0[6]  3[6]  0[6]  3[6] 
 4[6] | 0[6]  4[6]  2[6]  0[6]  4[6]  2[6] 
 5[6] | 0[6]  5[6]  4[6]  3[6]  2[6]  1[6] 

   -  | 0[6]  1[6]  2[6]  3[6]  4[6]  5[6] 
-------------------------------------------
 0[6] | 0[6]  5[6]  4[6]  3[6]  2[6]  1[6] 
 1[6] | 1[6]  0[6]  5[6]  4[6]  3[6]  2[6] 
 2[6] | 2[6]  1[6]  0[6]  5[6]  4[6]  3[6] 
 3[6] | 3[6]  2[6]  1[6]  0[6]  5[6]  4[6] 
 4

**Question 6 (puissance)**

> 1. Implémentez la fonction membre `__pow__(self, other)` qui prend en argument un entier `other` (celui-ci peut être négatif) et qui retourne l'élément à la puissance `other`. Cette fonction devra vérifier que `other` est bien un entier.
> 2. Dans l'anneau $\mathbb{Z}/12\mathbb{Z}$, calculez et affichez `a[12]^11$ pour $0\leq a<12$.
> 3. Faites de même avec $a[12]^(-11)$ en n'affichant les résultats que pour $a[12]$ inversible.